# Imports and data preparation

In [1]:
!git clone https://github.com/CSSEGISandData/COVID-19.git

Cloning into 'COVID-19'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 14864 (delta 2), reused 8 (delta 2), pack-reused 14853
Receiving objects: 100% (14864/14864), 47.67 MiB | 25.42 MiB/s, done.
Resolving deltas: 100% (7152/7152), done.


In [0]:
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import boxcox
from scipy.special import inv_boxcox
from fbprophet import Prophet

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
f_path = '../content/drive/My Drive/data/corona/main_df.csv'

In [0]:
for dir_nm,_,file_nm in os.walk('../content/COVID-19/csse_covid_19_data/csse_covid_19_time_series/'):
  for f in file_nm:
    if f.endswith('Confirmed.csv'):
      confirmed_data = pd.read_csv(os.path.join(dir_nm+f))
    elif f.endswith('Recovered.csv'):
      recover_data = pd.read_csv(os.path.join(dir_nm+f))
    elif f.endswith('Deaths.csv'):
      death_data = pd.read_csv(os.path.join(dir_nm+f))

In [0]:
confirmed_data = confirmed_data.melt(id_vars=['Province/State','Country/Region','Lat','Long'],var_name='Date',value_name='Confirmed')
recovered_data = recover_data.melt(id_vars=['Province/State','Country/Region','Lat','Long'],var_name='Date',value_name='Recovered')
death_data = death_data.melt(id_vars=['Province/State','Country/Region','Lat','Long'],var_name='Date',value_name='Deaths')

In [0]:
data = pd.concat([confirmed_data,death_data['Deaths'],recovered_data['Recovered']],axis=1).reset_index().drop(['index'],axis=1)

In [0]:
data[['Deaths','Recovered','Confirmed']] = data[['Deaths','Recovered','Confirmed']].apply(lambda s: s.fillna(0))

In [0]:
data["Date"] = data["Date"].apply(lambda s: pd.to_datetime(s).date())

In [10]:
data['Country/Region'].value_counts()

US                      14573
China                    1947
Canada                    649
Australia                 531
France                    531
                        ...  
United Arab Emirates       59
Holy See                   59
Martinique                 59
Liberia                    59
Kenya                      59
Name: Country/Region, Length: 162, dtype: int64

In [11]:
confirmed_data.dropna(subset=['Province/State'])['Country/Region'].value_counts()

US                14573
China              1947
Canada              649
Australia           531
France              531
United Kingdom      413
Netherlands         236
Denmark             177
Cruise Ship          59
Name: Country/Region, dtype: int64

In [0]:
data.to_csv('../content/drive/My Drive/data/corona/main_df.csv',index=False)

In [13]:
data.describe()

,Lat,Long,Confirmed,Deaths,Recovered
count,28143.000000,28143.000000,28143.000000,28143.000000,28143.000000
mean,30.965553,-34.570313,161.882457,5.494368,60.172903
std,19.365473,80.783759,2519.847218,109.294757,1346.659783
min,-41.454500,-157.858400,0.000000,0.000000,0.000000
25%,26.027500,-92.288400,0.000000,0.000000,0.000000
50%,37.648900,-74.053500,0.000000,0.000000,0.000000
75%,42.230200,21.745300,0.000000,0.000000,0.000000
max,71.706900,178.065000,67800.000000,4032.000000,58382.000000


In [0]:
data = pd.read_csv(f_path)

In [0]:
def rescale(values, new_min = 0, new_max = 127):
    output = []
    old_min, old_max = min(values), max(values)

    for v in values:
        new_v = (new_max - new_min) / (old_max - old_min) * (v - old_min) + new_min
        output.append(new_v)

    return output

# **WORLD**

In [0]:
latest = data['Date'].max()
bar_data = {}

bar_data['Confirmed'] = data[(data['Date'] == latest)].groupby(['Country/Region'])[data.columns[-3]].sum().reset_index().sort_values('Confirmed',ascending=False)
bar_data['deaths'] = data[(data['Date'] == latest)].groupby(['Country/Region'])[data.columns[-2]].sum().reset_index().sort_values('Deaths',ascending=False)
bar_data['recovered'] = data[(data['Date'] == latest)].groupby(['Country/Region'])[data.columns[-1]].sum().reset_index().sort_values('Recovered',ascending=False)

In [20]:
fig = make_subplots(rows=1,cols=1)

fig.add_trace(go.Bar(
    x=bar_data['Confirmed']['Country/Region'][:10],
    y=bar_data['Confirmed']['Confirmed'][:10],
    text=bar_data['Confirmed']['Confirmed'][:10],
    name='confirmed'),
    row=1,col=1
)

fig.add_trace(go.Bar(
    x=bar_data['deaths']['Country/Region'][:10],
    y=bar_data['deaths']['Deaths'][:10],
    text=bar_data['deaths']['Deaths'][:10],
    name='Deaths'),
    row=1,col=1
)

fig.add_trace(go.Bar(
    x=bar_data['recovered']['Country/Region'][:10],
    y=bar_data['recovered']['Recovered'][:10],
    text=bar_data['recovered']['Recovered'][:10],
    name='Recovered'),
    row=1,col=1
)

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(height=600,title='Cases reported on '+str(latest)+' Top 10 countries',showlegend=True)

fig.show()

# **US**

In [0]:
latest = data['Date'].max()
bar_data = {}

bar_data['Confirmed'] = data[(data['Date'] == latest) & (data['Country/Region'] == 'US')].groupby(['Province/State'])[data.columns[-3]].sum().reset_index().sort_values('Confirmed',ascending=False)
bar_data['deaths'] = data[(data['Date'] == latest) & (data['Country/Region'] == 'US')].groupby(['Province/State'])[data.columns[-2]].sum().reset_index().sort_values('Deaths',ascending=False)
bar_data['recovered'] = data[(data['Date'] == latest) & (data['Country/Region'] == 'US')].groupby(['Province/State'])[data.columns[-1]].sum().reset_index().sort_values('Recovered',ascending=False)

In [22]:
fig = make_subplots(rows=1,cols=1)

fig.add_trace(go.Bar(
    x=bar_data['Confirmed']['Province/State'][:10],
    y=bar_data['Confirmed']['Confirmed'][:10],
    text=bar_data['Confirmed']['Confirmed'][:10],
    name='confirmed'),
    row=1,col=1
)

fig.add_trace(go.Bar(
    x=bar_data['deaths']['Province/State'][:10],
    y=bar_data['deaths']['Deaths'][:10],
    text=bar_data['deaths']['Deaths'][:10],
    name='Deaths'),
    row=1,col=1
)

fig.add_trace(go.Bar(
    x=bar_data['recovered']['Province/State'][:10],
    y=bar_data['recovered']['Recovered'][:10],
    text=bar_data['recovered']['Recovered'][:10],
    name='Recovered'),
    row=1,col=1
)

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(height=600,title='Cases reported in US on '+str(latest)+' Top 10 Province/States',showlegend=True)

fig.show()

# **CHINA**

In [0]:
latest = data['Date'].max()
bar_data = {}

bar_data['Confirmed'] = data[(data['Date'] == latest) & (data['Country/Region'] == 'China')].groupby(['Province/State'])[data.columns[-3]].sum().reset_index().sort_values('Confirmed',ascending=False)
bar_data['deaths'] = data[(data['Date'] == latest) & (data['Country/Region'] == 'China')].groupby(['Province/State'])[data.columns[-2]].sum().reset_index().sort_values('Deaths',ascending=False)
bar_data['recovered'] = data[(data['Date'] == latest) & (data['Country/Region'] == 'China')].groupby(['Province/State'])[data.columns[-1]].sum().reset_index().sort_values('Recovered',ascending=False)

In [24]:
fig = make_subplots(rows=1,cols=1)

fig.add_trace(go.Bar(
    x=bar_data['Confirmed']['Province/State'][:10],
    y=bar_data['Confirmed']['Confirmed'][:10],
    text=bar_data['Confirmed']['Confirmed'][:10],
    name='confirmed'),
    row=1,col=1
)

fig.add_trace(go.Bar(
    x=bar_data['deaths']['Province/State'][:10],
    y=bar_data['deaths']['Deaths'][:10],
    text=bar_data['deaths']['Deaths'][:10],
    name='Deaths'),
    row=1,col=1
)

fig.add_trace(go.Bar(
    x=bar_data['recovered']['Province/State'][:10],
    y=bar_data['recovered']['Recovered'][:10],
    text=bar_data['recovered']['Recovered'][:10],
    name='Recovered'),
    row=1,col=1
)

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(height=600,title='Cases reported in China on '+str(latest)+' Top 10 Province/States',showlegend=True)

fig.show()

# **GERMANY**

In [0]:
latest = data['Date'].max()
bar_data = {}

bar_data['Confirmed'] = data[(data['Date'] == latest) & (data['Country/Region'] == 'Germany')].groupby(['Country/Region'])[data.columns[-3]].sum().reset_index().sort_values('Confirmed',ascending=False)
bar_data['deaths'] = data[(data['Date'] == latest) & (data['Country/Region'] == 'Germany')].groupby(['Country/Region'])[data.columns[-2]].sum().reset_index().sort_values('Deaths',ascending=False)
bar_data['recovered'] = data[(data['Date'] == latest) & (data['Country/Region'] == 'Germany')].groupby(['Country/Region'])[data.columns[-1]].sum().reset_index().sort_values('Recovered',ascending=False)

In [32]:
fig = make_subplots(rows=1,cols=1)

fig.add_trace(go.Bar(
    x=bar_data['Confirmed']['Country/Region'],
    y=bar_data['Confirmed']['Confirmed'],
    text=bar_data['Confirmed']['Confirmed'],
    name='confirmed'),
    row=1,col=1
)

fig.add_trace(go.Bar(
    x=bar_data['deaths']['Country/Region'],
    y=bar_data['deaths']['Deaths'],
    text=bar_data['deaths']['Deaths'],
    name='Deaths'),
    row=1,col=1
)

fig.add_trace(go.Bar(
    x=bar_data['recovered']['Country/Region'],
    y=bar_data['recovered']['Recovered'],
    text=bar_data['recovered']['Recovered'],
    name='Recovered'),
    row=1,col=1
)

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(height=600,title='Cases reported in Germany on '+str(latest),showlegend=True)

fig.show()

# **India**

In [0]:
latest = data['Date'].max()
bar_data = {}

bar_data['Confirmed'] = data[(data['Country/Region'] == 'India') & (data['Date'] == latest)].groupby(['Country/Region'])[data.columns[-3]].sum().reset_index().sort_values('Confirmed',ascending=False)
bar_data['deaths'] = data[(data['Country/Region'] == 'India') & (data['Date'] == latest)].groupby(['Country/Region'])[data.columns[-2]].sum().reset_index().sort_values('Deaths',ascending=False)
bar_data['recovered'] = data[(data['Country/Region'] == 'India') & (data['Date'] == latest)].groupby(['Country/Region'])[data.columns[-1]].sum().reset_index().sort_values('Recovered',ascending=False)

In [34]:
fig = make_subplots(rows=1,cols=1)

fig.add_trace(go.Bar(
    x=bar_data['Confirmed']['Country/Region'],
    y=bar_data['Confirmed']['Confirmed'],
    text=bar_data['Confirmed']['Confirmed'],
    name='confirmed'),
    row=1,col=1
)

fig.add_trace(go.Bar(
    x=bar_data['deaths']['Country/Region'],
    y=bar_data['deaths']['Deaths'],
    text=bar_data['deaths']['Deaths'],
    name='Deaths'),
    row=1,col=1
)

fig.add_trace(go.Bar(
    x=bar_data['recovered']['Country/Region'],
    y=bar_data['recovered']['Recovered'],
    text=bar_data['recovered']['Recovered'],
    name='Recovered'),
    row=1,col=1
)

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(height=600,title='Cases reported in Germany on '+str(latest),showlegend=True)

fig.show()

# **Austria**

In [0]:
latest = data['Date'].max()
bar_data = {}

bar_data['Confirmed'] = data[(data['Date'] == latest) & (data['Country/Region'] == 'Austria')].groupby(['Country/Region'])[data.columns[-3]].sum().reset_index().sort_values('Confirmed',ascending=False)
bar_data['deaths'] = data[(data['Date'] == latest) & (data['Country/Region'] == 'Austria')].groupby(['Country/Region'])[data.columns[-2]].sum().reset_index().sort_values('Deaths',ascending=False)
bar_data['recovered'] = data[(data['Date'] == latest) & (data['Country/Region'] == 'Austria')].groupby(['Country/Region'])[data.columns[-1]].sum().reset_index().sort_values('Recovered',ascending=False)

In [36]:
fig = make_subplots(rows=1,cols=1)

fig.add_trace(go.Bar(
    x=bar_data['Confirmed']['Country/Region'],
    y=bar_data['Confirmed']['Confirmed'],
    text=bar_data['Confirmed']['Confirmed'],
    name='confirmed'),
    row=1,col=1
)

fig.add_trace(go.Bar(
    x=bar_data['deaths']['Country/Region'],
    y=bar_data['deaths']['Deaths'],
    text=bar_data['deaths']['Deaths'],
    name='Deaths'),
    row=1,col=1
)

fig.add_trace(go.Bar(
    x=bar_data['recovered']['Country/Region'],
    y=bar_data['recovered']['Recovered'],
    text=bar_data['recovered']['Recovered'],
    name='Recovered'),
    row=1,col=1
)

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(height=600,title='Cases reported in Austria on '+str(latest),showlegend=True)

fig.show()

# **Confirmed** vs Recovered

**World**

In [37]:
f_data = data['Date'].unique().astype('str')
line_data = data.groupby(['Date']).agg({'Confirmed':'sum','Deaths':'sum','Recovered':'sum'}).reset_index()
fig = px.area()
fig.add_scatter(x=line_data['Date'],y=line_data['Confirmed'],mode='lines',name='Confirmed',fill='tozeroy')
fig.add_scatter(x=line_data['Date'],y=line_data['Recovered'],mode='lines',name='Recovered',fill='tozeroy',fillcolor='#9ec7be',line={'color':'#549283'})
fig.add_scatter(x=line_data['Date'],y=line_data['Deaths'],mode='lines',name='Deaths',fill='tozeroy',fillcolor='#e0b8c6',line={'color':'#bb5d7d'})
fig.update_layout(title_text='Confirmed vs Recovered cases of the world',showlegend=True)

**US**

In [38]:
f_data = data['Date'].unique().astype('str')
line_data = data[data['Country/Region'] == 'US'].groupby(['Date']).agg({'Confirmed':'sum','Deaths':'sum','Recovered':'sum'}).reset_index()
fig = px.area()
fig.add_scatter(x=line_data['Date'],y=line_data['Confirmed'],mode='lines',name='Confirmed',fill='tozeroy')
fig.add_scatter(x=line_data['Date'],y=line_data['Recovered'],mode='lines',name='Recovered',fill='tozeroy',fillcolor='#9ec7be',line={'color':'#549283'})
fig.add_scatter(x=line_data['Date'],y=line_data['Deaths'],mode='lines',name='Deaths',fill='tozeroy',fillcolor='#e0b8c6',line={'color':'#bb5d7d'})
fig.update_layout(title_text='Confirmed vs Recovered cases of US',showlegend=True)

**China**

In [39]:
f_data = data['Date'].unique().astype('str')
line_data = data[data['Country/Region'] == 'China'].groupby(['Date']).agg({'Confirmed':'sum','Deaths':'sum','Recovered':'sum'}).reset_index()
fig = px.area()
fig.add_scatter(x=line_data['Date'],y=line_data['Confirmed'],mode='lines',name='Confirmed',fill='tozeroy')
fig.add_scatter(x=line_data['Date'],y=line_data['Recovered'],mode='lines',name='Recovered',fill='tozeroy',fillcolor='#9ec7be',line={'color':'#549283'})
fig.add_scatter(x=line_data['Date'],y=line_data['Deaths'],mode='lines',name='Deaths',fill='tozeroy',fillcolor='#e0b8c6',line={'color':'#bb5d7d'})
fig.update_layout(title_text='Confirmed vs Recovered cases of China',showlegend=True)

**Germany**

In [40]:
f_data = data['Date'].unique().astype('str')
line_data = data[data['Country/Region'] == 'Germany'].groupby(['Date']).agg({'Confirmed':'sum','Deaths':'sum','Recovered':'sum'}).reset_index()
fig = px.area()
fig.add_scatter(x=line_data['Date'],y=line_data['Confirmed'],mode='lines',name='Confirmed',fill='tozeroy')
fig.add_scatter(x=line_data['Date'],y=line_data['Recovered'],mode='lines',name='Recovered',fill='tozeroy',fillcolor='#9ec7be',line={'color':'#549283'})
fig.add_scatter(x=line_data['Date'],y=line_data['Deaths'],mode='lines',name='Deaths',fill='tozeroy',fillcolor='#e0b8c6',line={'color':'#bb5d7d'})
fig.update_layout(title_text='Confirmed vs Recovered cases of Germany',showlegend=True)

**India**

In [41]:
f_data = data['Date'].unique().astype('str')
line_data = data[data['Country/Region'] == 'India'].groupby(['Date']).agg({'Confirmed':'sum','Deaths':'sum','Recovered':'sum'}).reset_index()
fig = px.area()
fig.add_scatter(x=f_data,y=line_data['Confirmed'],mode='lines',name='Confirmed',fill='tozeroy')
fig.add_scatter(x=line_data['Date'],y=line_data['Recovered'],mode='lines',name='Recovered',fill='tozeroy',fillcolor='#9ec7be',line={'color':'#549283'})
fig.add_scatter(x=line_data['Date'],y=line_data['Deaths'],mode='lines',name='Deaths',fill='tozeroy',fillcolor='#e0b8c6',line={'color':'#bb5d7d'})
fig.update_layout(title_text='Confirmed vs Recovered cases of India',showlegend=True)

**Austria**

In [42]:
f_data = data['Date'].unique().astype('str')
line_data = data[data['Country/Region'] == 'Austria'].groupby(['Date']).agg({'Confirmed':'sum','Deaths':'sum','Recovered':'sum'}).reset_index()

fig = px.area()
fig.add_scatter(x=f_data,y=line_data['Confirmed'],mode='lines',name='Confirmed',fill='tozeroy')
fig.add_scatter(x=line_data['Date'],y=line_data['Recovered'],mode='lines',name='Recovered',fill='tozeroy',fillcolor='#9ec7be',line={'color':'#549283'})
fig.add_scatter(x=line_data['Date'],y=line_data['Deaths'],mode='lines',name='Deaths',fill='tozeroy',fillcolor='#e0b8c6',line={'color':'#bb5d7d'})
fig.update_layout(title_text='Confirmed vs Recovered cases of India',showlegend=True)

# World data confirmed cases geoplot

In [0]:
time_data = data.copy()
time_data['scaled_confirmed'] = time_data['Confirmed'].apply(lambda s : np.log(s))
time_data['scaled_confirmed'] = time_data['scaled_confirmed'].apply(lambda s:0 if s == -np.inf else s)
time_data['Date'] =  time_data['Date'].astype(str)

In [44]:
fig = px.scatter_geo(time_data,lat='Lat',lon='Long',size='scaled_confirmed',opacity=0.5,
                     hover_name='Country/Region', animation_frame='Date',color='Confirmed',
                     color_continuous_scale=px.colors.cmocean.tempo_r,
                     hover_data=['Confirmed'],projection='natural earth')
fig.update_geos(showcountries=True,showland=True,showocean=True,oceancolor='#66c2ff',landcolor='#adebad')

# Confirmed rate, Death rate, Recovery rate

In [55]:
recovery_data = data.copy()
recovery_data.describe(include='all')

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
count,19116,28143,28143.000000,28143.000000,28143,28143.000000,28143.000000,28143.000000
unique,322,162,NaN,NaN,59,NaN,NaN,NaN
top,Diamond Princess,US,NaN,NaN,2020-03-19,NaN,NaN,NaN
freq,118,14573,NaN,NaN,477,NaN,NaN,NaN
mean,NaN,NaN,30.965553,-34.570313,NaN,161.882457,5.494368,60.172903
std,NaN,NaN,19.365473,80.783759,NaN,2519.847218,109.294757,1346.659783
min,NaN,NaN,-41.454500,-157.858400,NaN,0.000000,0.000000,0.000000
25%,NaN,NaN,26.027500,-92.288400,NaN,0.000000,0.000000,0.000000
50%,NaN,NaN,37.648900,-74.053500,NaN,0.000000,0.000000,0.000000
75%,NaN,NaN,42.230200,21.745300,NaN,0.000000,0.000000,0.000000


In [53]:
countries = np.unique(recovery_data['Country/Region'])

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas, The', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Benin', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Brazil', 'Brunei', 'Bulgaria',
       'Burkina Faso', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Cruise Ship', 'Cuba', 'Cyprus',
       'Czechia', 'Denmark', 'Djibouti', 'Dominican Republic', 'Ecuador',
       'Egypt', 'El Salvador', 'Equatorial Guinea', 'Estonia', 'Eswatini',
       'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia, The',
       'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala', 'Guinea',
       'Guyana', 'Haiti', 'Holy See', 'Honduras', 'Hungary', 'Iceland',
       'India', 'In

In [0]:
conf_rate = {}
recover_rate = {}
death_rate = {}
for c in countries:
  country_data = recovery_data[recovery_data['Country/Region'] == c].groupby(['Date']).agg({'Confirmed':'sum','Recovered':'sum','Deaths':'sum'}).reset_index()
  c_lis = []
  r_lis = []
  d_lis = []
  
  for i in range(1,len(country_data['Confirmed'])):
    c_lis.append(0 if country_data['Confirmed'][i-1] == 0  else country_data['Confirmed'][i] / country_data['Confirmed'][i-1])
    r_lis.append(0 if country_data['Recovered'][i-1] == 0  else country_data['Recovered'][i] / country_data['Recovered'][i-1])
    d_lis.append(0 if country_data['Deaths'][i-1] == 0  else country_data['Deaths'][i] / country_data['Deaths'][i-1])
  conf_rate[c] = np.mean(c_lis)
  recover_rate[c] = np.mean(r_lis)
  death_rate[c] = np.mean(d_lis)
  print(c,'done')



In [0]:
import collections
sorted_conf = collections.OrderedDict(sorted(conf_rate.items() , key=lambda kv:kv[1],reverse=True))
sorted_death = collections.OrderedDict(sorted(recover_rate.items() , key=lambda kv:kv[1],reverse=True))
sorted_recover = collections.OrderedDict(sorted(death_rate.items() , key=lambda kv:kv[1],reverse=True))

**World top 10 countries**

In [85]:
fig = make_subplots(rows=1,cols=1)

fig.add_trace(go.Bar(
    x=list(sorted_conf.keys())[:10],
    y=list(sorted_conf.values())[:10],
    text=list(sorted_conf.values())[:10],
    name='confirmed rate'),
    row=1,col=1
)

fig.add_trace(go.Bar(
    x=list(sorted_death.keys())[:10],
    y=list(sorted_death.values())[:10],
    text=list(sorted_death.values())[:10],
    name='Death rate'),
    row=1,col=1
)

fig.add_trace(go.Bar(
    x=list(sorted_recover.keys())[:10],
    y=list(sorted_recover.values())[:10],
    text=list(sorted_recover.values())[:10],
    name='Recover rate'),
    row=1,col=1
)

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(height=600,width=2000,title='Confirmed rate, Death rate, Recovery rate',showlegend=True)

fig.show()

**US**

In [99]:
fig = make_subplots(rows=1,cols=1)

fig.add_trace(go.Bar(
    x=['US'],
    y=[conf_rate['US']],
    text=[conf_rate['US']],
    name='confirmed rate'),
    row=1,col=1
)

fig.add_trace(go.Bar(
    x=['US'],
    y=[death_rate['US']],
    text=[death_rate['US']],
    name='Death rate'),
    row=1,col=1
)

fig.add_trace(go.Bar(
    x=['US'],
    y=[recover_rate['US']],
    text=[recover_rate['US']],
    name='Recover rate'),
    row=1,col=1
)

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(height=600,title='Confirmed rate, Death rate, Recovery rate of US',showlegend=True)

fig.show()

**India**

In [100]:
fig = make_subplots(rows=1,cols=1)

fig.add_trace(go.Bar(
    x=['India'],
    y=[conf_rate['India']],
    text=[conf_rate['India']],
    name='confirmed rate'),
    row=1,col=1
)

fig.add_trace(go.Bar(
    x=['India'],
    y=[death_rate['India']],
    text=[death_rate['India']],
    name='Death rate'),
    row=1,col=1
)

fig.add_trace(go.Bar(
    x=['India'],
    y=[recover_rate['India']],
    text=[recover_rate['India']],
    name='Recover rate'),
    row=1,col=1
)

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(height=600,title='Confirmed rate, Death rate, Recovery rate of India',showlegend=True)

fig.show()

**China**

In [101]:
fig = make_subplots(rows=1,cols=1)

fig.add_trace(go.Bar(
    x=['China'],
    y=[conf_rate['China']],
    text=[conf_rate['China']],
    name='confirmed rate'),
    row=1,col=1
)

fig.add_trace(go.Bar(
    x=['China'],
    y=[death_rate['China']],
    text=[death_rate['China']],
    name='Death rate'),
    row=1,col=1
)

fig.add_trace(go.Bar(
    x=['China'],
    y=[recover_rate['China']],
    text=[recover_rate['China']],
    name='Recover rate'),
    row=1,col=1
)

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(height=600,title='Confirmed rate, Death rate, Recovery rate of China',showlegend=True)

fig.show()

**Austria**

In [102]:
fig = make_subplots(rows=1,cols=1)

fig.add_trace(go.Bar(
    x=['Austria'],
    y=[conf_rate['Austria']],
    text=[conf_rate['Austria']],
    name='confirmed rate'),
    row=1,col=1
)

fig.add_trace(go.Bar(
    x=['Austria'],
    y=[death_rate['Austria']],
    text=[death_rate['Austria']],
    name='Death rate'),
    row=1,col=1
)

fig.add_trace(go.Bar(
    x=['Austria'],
    y=[recover_rate['Austria']],
    text=[recover_rate['Austria']],
    name='Recover rate'),
    row=1,col=1
)

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(height=600,title='Confirmed rate, Death rate, Recovery rate of Austria',showlegend=True)

fig.show()

# **References**
1) https://towardsdatascience.com/visualizing-coronavirus-outbreak-da835608fb88<br>
2) https://mode.com/example-gallery/forecasting_prophet_python_cookbook/<br> 
3) dataset = https://github.com/CSSEGISandData/COVID-19


In [104]:
!git remote add origin https://github.com/JShyam33/Corona-data-visualization.git
!git push -u origin master


fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
